# Análisis de Sentimiento (machine learning)

## 1. Normalizar (limpiar los datos)

In [109]:
# coding=utf-8

import csv
import re
import string
from functools import reduce
from unicodedata import normalize
import unicodedata

def cargar_corpus(nombre_archivo, debug = False):
    """
        el archivo de entrada csv contiene dos columnas separador
        por el caracter: |
        primera columna -> polaridad texto (positivo o negativo)
        segunda columna -> texto
    """
    texto = []
    polaridad = []
    x = ""
    with open(nombre_archivo, encoding='utf-8') as csv_file:
        reader = csv.reader(csv_file, delimiter='|')
        for row in reader:
            if debug:
                print(row[0],row[1])
            polaridad.append(row[0])
            texto.append(row[1])
    return polaridad, texto

def guardar_corpus_csv(nombre_archivo, texto):
    """Guarda el archivo de texto pre-procesado, se utiliza para normalizar corpus"""
    csv_salida = open(nombre_archivo, 'w', newline='', encoding='utf-8')
    salida = csv.writer(csv_salida, delimiter='|')
    salida.writerows(texto)
    csv_salida.close()

def cargar_diccionario(nombre_archivo):
    """Carga un diccionario slang o de emoticons"""
    modismos = {}
    with open(nombre_archivo, encoding='utf-8') as csv_file:
        reader = csv.reader(csv_file, delimiter='\t')
        for row in reader:
            modismos[row[0]] = row[1] #slang/emoticon-significado
    return modismos

def traducir_emoticons(texto):
    """Dado un texto detecta los emoticons y los traduce a un texto equivalente"""
    EMOTICONS = cargar_diccionario('corpus/emoticons.csv')

    words = texto.split()
    reformed = [EMOTICONS[word] if word in EMOTICONS else word for word in words]
    reformed = ' '.join(reformed)
    return reformed

def traducir_slang(texto):
    """Dado un texto detecta los términos slang y los traduce a un texto equivalente"""
    SLANG = cargar_diccionario('corpus/slang.csv')
    words = texto.split()
    reformed = [SLANG[word] if word in SLANG else word for word in words]
    reformed = ' '.join(reformed)
    return reformed

def elimina_tildes(cadena):
    """Eliminar acentos en una cadena de texto"""
    return ''.join((c for c in unicodedata.normalize('NFD', cadena) if unicodedata.category(c) != 'Mn'))

def remover_numeros(texto):
    """Eliminar numeros en una cadena de texto  """
    return re.sub(r'[0-9]*', '', texto)

def remover_chars(texto):
    """Eliminar algunos chars de una cadena de texto"""
    return re.sub(r'["-:¡!¿?$.,;“”€ª]','',texto)
    
# elimina letras repetidas, ejemplo: siiii, sii --> si
def remover_letras(texto):
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", texto)

# Realiza un preprocesamiento del corpus antes de iniciar
# la tokenizacion, traduce emoticons y otros slangs a texto
def normalizar_texto(corpus):
    polaridad, texto = cargar_corpus(corpus, debug = True)
    indice = 0
    texto_preprocesado = []
    
    for frase in texto:
        preprocess = traducir_emoticons(frase).lower()
        preprocess = traducir_slang(preprocess)
        preprocess = elimina_tildes(frase)
        preprocess = remover_numeros(preprocess)
        preprocess = remover_chars(preprocess)
        
        texto_preprocesado.append((polaridad[indice], preprocess.strip()))
        indice += 1
    
    return texto_preprocesado

In [110]:
def normalizar_corpus(fuente, destino):
    print("Normalizando archivo:" + fuente)
    texto = normalizar_texto(fuente)
    guardar_corpus_csv(destino, texto)
    print("Archivo normalizado:" + destino)

In [111]:
def test():
    normalizar_corpus('corpus/corpus_prog.csv','corpus/corpus_ok2.csv')

In [112]:
test()

Normalizando archivo:corpus/corpus_prog.csv
negativo el sistema no funciona porque no lo puedo probar todo
negativo programar es como el sexo un único error y tienes que estar soportándolo toda la vida
positivo no es por enamorarte pero soy programador con maestría en ciencias de la computación
positivo se que suena un poco egocentrico pero yo no soy perfecto mi codigo si
positivo primero resuelve el problema entonces escribe el código
positivo si leer un código que escribiste hace más de dos semanas es como si lo vieras por primera vez
positivo Estoy aprendiendo a programar las pilas, colas, hash tables, listas y demás estructura de datos
positivo Ya se como sacar la lista de atributos de un arraylist de objetos
negativo Necesito ayuda con mi calculadora en netbeans y java necesito de su ayuda urgentemente de otra forma voy a reprobar la materia
positivo Ya le estoy entendiendo a la programación avanzada en Java estoy emocionado
negativo Necesito ayuda para saber como se llena un JCom

negativo y es cuando te comprometes con la programacion que comienzas a odiar windows cara triste
negativo me caga la gente que solo publica que tiene que estudiar que sus respuestas son no tienes que estudiar osea pudranse
negativo quisiera que mis amanecidas no fueran por estudiar a esta noche la llamare supervivencia
positivo deje dreamweaver con mil codigos abierto por mas de  horas se acabo la pila no lo guarde y todo esta como lo deje por eso te amo mac
negativo enserio me cae gordo que los profes nos estresen tanto en estas fechas
negativo segun le yo estoy estudiando para mi examen de egreso de la universidad y me estoy quedando dormida
negativo no mereces que te pasen la tarea de mate discretas
positivo mandame energias mejor que tengo mucho q estudiar estoy haciendo un gran esfuerzo donde vergas estas concentracion
negativo cuando sientas estres por tus finales recuerda que yo no he hecho tercer parcial
negativo a veces yo solita me complico la vida tanto academicamente socia

## 2. Entrenar el corpus en base a un algoritmo clasificador de machine learning

In [116]:
import sys
import nltk
from sklearn import linear_model
from sklearn import svm
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from string import punctuation
import _pickle as pickle
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB

# Si es la primera vez se tienen que descarcar los stopwords de:
nltk.download("stopwords")
spanish_stopwords = stopwords.words('spanish')
stemmer = SnowballStemmer('spanish')
non_words = list(punctuation)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\oramas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [117]:
# Stemming: transformamos cada palabra en su raiz.
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

# Tokenizar, este paso convierte una cadena de texto en una lista de palabras (tokens).
# Mediante el uso de nltk.word_tokenize), remueve signos de puntuación y acentos.
def tokenize(texto):
    texto = ''.join([c for c in texto if c not in non_words])
    tokens = word_tokenize(texto)
    # stem
    try:
        stems = stem_tokens(tokens, stemmer)
    except Exception as e:
        print(e)
        print(texto)
        stems = ['']
    return stems

# Definición del vector de caracteristicas
count_vectorizer = CountVectorizer(
    analyzer='word',
    tokenizer=None,
    lowercase=True,
    stop_words=spanish_stopwords
)

# Crea el pipeline junto con el clasificador
text_clf = Pipeline([('vect', count_vectorizer),
                     ('tfidf', TfidfTransformer(use_idf=False)),
                     ('clf', BernoulliNB()),
                     ])


In [118]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

def evaluar_modelo():
    print('("******************* EVALUAR CORPUS *******************')    
    polaridad, texto = cargar_corpus('corpus/corpus_ok2.csv')
    # preparing data for split validation. 60% training, 40% test
    X_train, X_test, y_train, y_test = train_test_split(texto, polaridad, test_size=0.4, random_state=43)

    classifier = text_clf.fit(X_train,y_train)
    print("Score:",classifier.score(X_test,y_test))
    predicted = classifier.predict(X_test)
    print (classification_report(y_test,predicted))
    print ("Nivel de predicción {:.2%}".format(accuracy_score(y_test,predicted)))

In [119]:
evaluar_modelo()

("******************* EVALUAR CORPUS *******************
Score: 0.710084033613
             precision    recall  f1-score   support

   negativo       0.68      0.84      0.75       126
   positivo       0.76      0.56      0.65       112

avg / total       0.72      0.71      0.70       238

Nivel de predicción 71.01%


In [120]:
# Preprocess crear term frequency matrix CountVectorizer convierte la columna de texto en una matriz 
# en la que cada palabra es una columna
# cuyo valor es el número de veces que dicha palabra aparece en cada texto. 
# Convierte a minusculas elimina signos de puntuacion y remueve stopwords
# Entrena el corpus y lo guarda como un objeto serializado.
def entrenar_clasificador():
    polaridad,texto = cargar_corpus('corpus/corpus_prog.csv')
    text_clf.fit(texto, polaridad)
    modelo = open("corpus/corpus_prog.m","wb")
    pickle.dump(text_clf, modelo)
    modelo.close()

In [121]:
# Dado un texto predice su sentimiento
def obtener_sentimiento(texto):
    modelo = open("corpus/corpus_prog.m", "rb")
    text_clf2 = pickle.load(modelo)
    modelo.close()

    newTexto = [(texto)]
    sentimiento = text_clf2.predict(newTexto)
    return(sentimiento)

In [122]:
def testing():
    print("**************************************** TRAINING CORPUS ******************************************")
    # Si es la primera vez se tiene que entrenar el corpus, después ya no es necesario
    entrenar_clasificador()
    print('me cansa estar viendo los mismos temaz = ',obtener_sentimiento('me cansa estar viendo los mismos temaz'))
    print('me desespera no poder entender al tema =', obtener_sentimiento('me desespera no poder entender al tema'))
    print('Programar en Java es divertido =', obtener_sentimiento('Programar en Java es divertido'))
    print('Me gusto el area en el que estoy, seguire trabajando en el proyecto =', obtener_sentimiento('Me gusto el area en el que estoy, seguire trabajando en el proyecto'))
    print("***********************************************************")


In [123]:
testing()

**************************************** TRAINING CORPUS ******************************************
me cansa estar viendo los mismos temaz =  ['negativo']
me desespera no poder entender al tema = ['negativo']
Programar en Java es divertido = ['positivo']
Me gusto el area en el que estoy, seguire trabajando en el proyecto = ['positivo']
***********************************************************
